In [8]:
#Whisper speech-to-Text

In [9]:
#needed to install some things to make import feasible

import torch
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import load_dataset


In [10]:
#load model and processor
processor = WhisperProcessor.from_pretrained("openai/whisper-tiny")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")
model.config.forced_decoder_ids = None

In [ ]:
ds = load_dataset("common_voice", "fr", split="test", streaming=True)
ds = ds.cast_column("audio", Audio(sampling_rate=16_000))
input_speech = next(iter(ds))["audio"]
input_features = processor(input_speech["array"], sampling_rate=input_speech["sampling_rate"], return_tensors="pt").input_features

In [6]:
#initializes model
model_id = "openai/whisper-tiny"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

NameError: name 'AutoModelForSpeechSeq2Seq' is not defined

In [ ]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import load_dataset

# load model and processor
processor = WhisperProcessor.from_pretrained("openai/whisper-tiny")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")
model.config.forced_decoder_ids = None

# load dummy dataset and read audio files
ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")
sample = ds[0]["audio"]
input_features = processor(sample["array"], sampling_rate=sample["sampling_rate"], return_tensors="pt").input_features

# generate token ids
predicted_ids = model.generate(input_features)
# decode token ids to text

transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)


/Users/jenniferkiunke/.pyenv/versions/3.10.6/envs/pocketcoach/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [ ]:
transcription

[' Mr. Quilter is the apostle of the middle classes and we are glad to welcome his gospel.']

In [7]:
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)

NameError: name 'predicted_ids' is not defined

In [ ]:
#build pipeline for speech-to-text
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
)

In [ ]:
#we need to load audio files online for a test
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq
import librosa # Import librosa to load audio from URL
import soundfile as sf # Often useful for robust audio loading, but librosa.load can handle URLs

In [ ]:
#load dataset and get a sample audio input
dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
sample = dataset[0]["audio"]

In [ ]:
print(f"Original audio duration: {len(full_audio_array) / sampling_rate:.2f} seconds")
print(f"Using a sliced audio duration: {len(short_audio_array) / sampling_rate:.2f} seconds")


In [ ]:
result = pipe(sample)
print(result["text"])